In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import re
import string
import collections
from os.path import exists
import time
pd.set_option('display.max_rows', 500)

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [3]:
# path = '../Data/CorporaData'
# raw = '../OutputRaw'

In [4]:
# files = []
# for folder in os.listdir(path):    
#     if folder != '.DS_Store' and folder != '.ipynb_checkpoints':
#         for file in os.listdir(os.path.join(path, folder)):
#             if file != '.ipynb_checkpoints':
#                 search_key = '{}_{}'.format(file.split('.')[0], file.split('.')[2])
#                 for raw_folder in os.listdir(raw):
#                     if raw_folder != '.DS_Store' and raw_folder != '.ipynb_checkpoints':
#                         for raw_file in os.listdir(os.path.join(raw, raw_folder)):
#                             if raw_file != '.ipynb_checkpoints':
#                                 if search_key == '{}_{}'.format(raw_file.split('_')[0], raw_file.split('_')[1]):
#                                     files.append((os.path.join(path, os.path.join(folder, file)), os.path.join(raw, os.path.join(raw_folder, raw_file))))

In [5]:
# dataframes = []
# for tpl in files:
#     if exists('../OutputRawText/{}/{}'.format(tpl[1].split('/')[2], tpl[1].split('/')[3])):
#         continue
#     else:
#         start = time.time()
#         output_raw = pd.read_csv(tpl[1])
#         corpora = pd.read_csv(tpl[0])
#         corpora.columns = ['URL', 'Title', 'RawText']
#         merged = pd.merge(output_raw, corpora, on='URL')
#         merged['RawText_preprocessed'] = merged['RawText'].apply(lambda text: str(text).split('TOPICS: TOPIC FREQUENCY ')[0])
#         merged['RawText_preprocessed'] = merged['RawText_preprocessed'].apply(lambda text: str(text)[str(text).find('[[TITLE.END]] ') + len('[[TITLE.END]] '):])   
#         merged['RawText_preprocessed'] = merged['RawText_preprocessed'].apply(lambda text: ' '.join(re.sub('<[^<]+?>', '', word) for word in text.split()))   
#         merged['RawText_preprocessed'] = merged['RawText_preprocessed'].apply(lambda text: text.translate(translator)) 
#         dataframes.append((tpl[1], merged))
#         merged.to_csv('../OutputRawText/{}/{}'.format(tpl[1].split('/')[2], tpl[1].split('/')[3]), index=False)
#         end = time.time()
#         print('File {} done in {} seconds'.format(tpl[0], end-start))

In [6]:
path = '../OutputRaw2'

In [7]:
files = []
for folder in os.listdir(path):    
    if folder != '.DS_Store' and folder != '.ipynb_checkpoints':
        for file in os.listdir(os.path.join(path, folder)):
            if file != '.ipynb_checkpoints':
                files.append(os.path.join(path, os.path.join(folder, file)))

In [8]:
# for merged in files:
#     df = pd.read_csv(merged)
#     start = time.time()
#     df['Matched Firms'] = None
#     for i in range(len(df.index)):
#         loi = []
#         if str(df.loc[i, 'Hits']) != 'No Hits':
#             for firm in df.loc[i, 'Hits'].split(', '):
#                 og = firm.split('(')[1].replace(')', '').strip()
#                 for segment in str(df.loc[i, 'RawText_preprocessed']).split('TIME START'):
#                     if ' {} '.format(firm.split('(')[0].strip()) in segment.lower():
#                         loi.append((firm.split('(')[0].strip(), segment.split('TIME END')[0].strip(), og))
#         df.loc[i, 'Matched Firms'] = ', '.join(['{}/{}/{}'.format(tpl[0], tpl[1], tpl[2]) for tpl in loi])
#     del df['Matched Organizations']
#     del df['Hits']
#     del df['Hits Frequency']
#     del df['Validated']
#     df.to_csv(merged, index=False)
#     end = time.time()
#     print('Time Taken to finish {}: {} seconds'.format(merged, end-start))

In [9]:
# dfs = []
# for file in files:
#     df = pd.read_csv(file)
#     df['Hits Frequency'] = None
#     for i in range(len(df.index)):
#         loi = []
#         if df.loc[i, 'Matched Firms'] != 'No Hits':
#             for firm_name in str(df.loc[i, 'Matched Firms']).split(', '):
#                 if firm_name != 'nan':
#                     try:
#                         loi.append((firm_name.split('/')[0], firm_name.split('/')[2]))
#                     except:
#                         print(type(firm_name))
#         else:
#             continue
#         df.loc[i, 'Hits Frequency'] = ', '.join('{}: {}'.format(str(k).replace("'", ''), v) for k,v in collections.Counter(loi).items())
#     df.to_csv(file, index=False)

In [14]:
final_dataframes = []
for file in files:
    if file.split('/')[2] == 'Bloomberg':
        start=time.time()
        if exists('../OutputRawFormatted/{}/{}'.format(file.split('/')[2], file.split('/')[3])):
            continue
        else:
            df = pd.read_csv(files[0])
            lot = []
            for i in range(len(df.index)):
                if df.loc[i, 'Matched Firms'] != 'No Hits':
                    for firm_name in str(df.loc[i, 'Matched Firms']).split(', '):
                        if firm_name != 'nan':
                            key = '/'.join(firm_name.split('/')[1:2])
                            lot.append((df.loc[i, 'URL'],
                                        df.loc[i, 'Date'],
                                        df.loc[i, 'Title'],
                                        firm_name.split('/')[0], 
                                        firm_name.split('/')[1], 
                                        firm_name.split('/')[2],
                                        df.loc[i, 'Context Window'],
                                        df.loc[i, 'Hits Frequency'],
                                        df.loc[i, 'Matched Firms'].count(key)
                                       ))
                else:
                    continue

            final_df = pd.DataFrame(lot, columns = ['URL', 'Date', 'Title', 'Search Term', 'Time', 'Firm Name', 'Context Window', 'Hits Frequency', 'Frequency'])
            final_df['Time'] = final_df['Time'].apply(lambda t: '{}:{} {}'.format(t.split()[0], t.split()[1], t.split()[2])) 
            final_dataframes.append(final_df)
            end=time.time()
            print('File {} took {} seconds'.format(file, end-start))

File ../OutputRaw/Bloomberg/Bloomberg_2016_hits.csv took 7.923048973083496 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2017_hits.csv took 7.900598049163818 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2013_hits.csv took 7.846573114395142 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2018_hits.csv took 7.928839206695557 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2019_hits.csv took 7.8592588901519775 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2015_hits.csv took 7.952701091766357 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2014_hits.csv took 7.944279909133911 seconds
File ../OutputRaw/Bloomberg/Bloomberg_2020_hits.csv took 7.939279079437256 seconds


In [18]:
final_dataframes[0]['Hits Frequency'][0]

'(audience, AUDIENCE INC): 1, (facebook, FACEBOOK INC): 1, (west, WEST CORP): 1, (twitter, TWITTER INC): 1, (now, NOW INC): 22, (joint, JOINT CORP): 1, (box, BOX INC): 1, (planet fitness, PLANET FITNESS INC): 1, (square, SQUARE INC): 24, (line, LINE CORP): 3, (snap, SNAP INC): 1, (best, BEST INC): 13, (southern, SOUTHERN CO): 1, (fox, FOX CORP): 11, (dow, DOW INC): 2, (health, S C HEALTH CORP): 1, (courier, COURIER CORP): 1, (eastern, EASTERN CO): 1, (energy, NV ENERGY INC): 1, (ball, BALL CORP): 2, (unit, UNIT CORP): 1, (team, TEAM INC): 1, (pool, POOL CORP): 3, (waters, WATERS CORP): 2, (guess, GUESS INC): 3, (aware, AWARE INC): 1, (orange, ORANGE): 1, (sleep number, SLEEP NUMBER CORP): 1, (move, MOVE INC): 1, (netflix, NETFLIX INC): 1'

In [19]:
final_dataframes[0]['Frequency'][0]

2

In [21]:
if ' now ' in 'I am now coming around now to do it now':
    print('1')

1


In [ ]:
for df in final_dataframes:
    ot = []
    for i in range(len(df.index)):
        for item in df.loc[i, 'Hits Frequency'].split('('):
            if not item:
                continue
            else:
                key = item.split(', ')[0]
                if key == df.loc[i, 'Search Term']:
                    try:
                        df.loc[i, 'Hits Frequency'] = item.split(', ')[1].split(': ')[1]
                    except:
                        print(item, i)
    print(df['Hits Frequency'][0])
    break